<a href="https://colab.research.google.com/github/gr3ybr0w/cookbook/blob/master/Machine_Learning/Neural_Networks/TensorFlow/Keras/keras_wide_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this model directly in the browser with zero setup, open it in [Colab here](https://colab.research.google.com/github/sararob/keras-wine-model/blob/master/keras-wide-deep.ipynb).

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [0]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.12.0


In [0]:
# Get the data: original source is here: https://www.kaggle.com/zynicide/wine-reviews/data
URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)


49790976/49783940 [==============================] - 1s 0us/step


In [0]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

In [0]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
52565,52565,France,"This is powerful, typical of Les Preuses. Whit...",Les Preuses Grand Cru,94,67.0,Burgundy,Chablis,NaN,Chardonnay,La Chablisienne
13451,13451,US,Nice aromatics on this Châteauneuf-du-Pape-ins...,Fleur De Montagne,86,24.0,California,El Dorado,Sierra Foothills,Rhône-style Red Blend,Sierra Vista
113108,113108,US,There's lots of harsh acidity and tannins in t...,Ashley's Vineyard,84,50.0,California,Sta. Rita Hills,Central Coast,Pinot Noir,Fess Parker
139525,139525,France,"Rich and ripe, with buttery, toasty aromas tha...",Les Folatiéres Premier Cru,92,100.0,Burgundy,Puligny-Montrachet,NaN,Chardonnay,Lucien Le Moine
1928,1928,US,"This Merlot is dry, full-bodied and delicious ...",Grand Reserve,90,28.0,California,Sonoma County,Sonoma,Merlot,Kendall-Jackson


In [0]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns[0], axis=1) 

variety_threshold = 500 # Anything that occurs less than this will be removed.
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]

In [0]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 95646
Test size: 23912


In [0]:
# Train features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

# Train labels
labels_train = data['price'][:train_size]

# Test features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]

# Test labels
labels_test = data['price'][train_size:]

In [0]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 12000 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

In [0]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [0]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train) + 1

# Convert labels to one hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

In [0]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [0]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          3082496     concatenate[0][0]                
__________

In [0]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [0]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 170)               0         
_________________________________________________________________
embedding (Embedding)        (None, 170, 8)            96000     
_________________________________________________________________
flatten (Flatten)            (None, 1360)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [0]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 170)          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________

In [0]:
# Run training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
95646/95646 [==============================] - 59s 614us/step - loss: 1066.0892 - acc: 0.0000e+00
Epoch 2/10
95646/95646 [==============================] - 59s 614us/step - loss: 815.8045 - acc: 0.0000e+00
Epoch 3/10
95646/95646 [==============================] - 59s 616us/step - loss: 687.1002 - acc: 0.0000e+00
Epoch 4/10
95646/95646 [==============================] - 58s 608us/step - loss: 572.2075 - acc: 0.0000e+00
Epoch 5/10
95646/95646 [==============================] - 58s 605us/step - loss: 462.2863 - acc: 0.0000e+00
Epoch 6/10
95646/95646 [==============================] - 57s 601us/step - loss: 361.2607 - acc: 0.0000e+00
Epoch 7/10
95646/95646 [==============================] - 58s 603us/step - loss: 268.4163 - acc: 0.0000e+00
Epoch 8/10
95646/95646 [==============================] - 57s 595us/step - loss: 197.0382 - acc: 0.0000e+00
Epoch 9/10
95646/95646 [==============================] - 58s 607us/step - loss: 143.1778 - acc: 0.0000e+00
Epoch 10/10
95646/95646 [==

In [0]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, batch_size=128)

23912/23912 [==============================] - 4s 176us/step


[916.14222349216, 0.0]

In [0]:
# Generate predictions
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [0]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

Spicy strawberries on the nose lead straight to a full-bodied, totally balanced and healthy palate that is eager to deliver zesty raspberry and tea-like flavors. Not hard to enjoy, with peppery rosé tannins and good length. Freshness is key to this wine, and as of June 2007 it was drinking perfectly. Imported by Freixenet USA.
Predicted:  17.279507 Actual:  14.0 

A fresh, ripe wine that tastes rich enough to be a beerenauslese. It is packed with honey flavors, baked apples, mulberries and light acidity. At 7.5% alcohol, you can give into the temptation to drink a second glass. Imported by Michael Skurnik Wines.
Predicted:  41.9755 Actual:  66.0 

This is a small production wine blended by French sommeliers. The result is a soft, rich wine, deliciously ripe and perfumed. There are touches of honey as well as yellow fruits and mango juice. The acidity gives the wine a lift and makes a contrast to the richness of the wine. Drink now.
Predicted:  41.2355 Actual:  46.0 

Soft scents sugges

In [0]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  7.148534440994263
